In [30]:
!7z x training.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 619776 bytes (606 KiB)

Extracting archive: training.7z
--
Path = training.7z
Type = 7z
Physical Size = 619776
Headers Size = 3396
Method = LZMA2:768k
Solid = +
Blocks = 1

  0%    
Would you like to replace the existing file:
  Path:     ./training/кроссовки/0.jpg
  Size:     1405 bytes (2 KiB)
  Modified: 2019-12-04 19:04:06
with the file from archive:
  Path:     training/кроссовки/0.jpg
  Size:     1405 bytes (2 KiB)
  Modified: 2019-12-04 19:04:06
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? A

  0% 4 - training/кроссовки/0.jpg                                          Everything is Ok

Folders: 4


In [5]:
import numpy, cv2
import os
import numpy as np

# Размер входного изображения
size = 256

# Папка с обучающей выборкой
train_dir = "training"

# Цвет изображения
CLR = cv2.IMREAD_COLOR #COLOR GRAYSCALE

# Количество классов
classes = 3

def create_train_data():
  # Массивы X и Y
  train_data = []
  train_labels = []
  
  #Индекс класса
  pp = 0
  
  # По всем классам(папкам), и по всем файлам в них
  for p1 in sorted(os.listdir(train_dir)):
    for img in os.listdir(os.path.join(train_dir, p1)):
          # Формируем путь к файлу
          path = os.path.join(os.path.join(train_dir, p1), img)
          # Изменяем размер изображения и цветовое пространство
          img = cv2.resize(cv2.imread(path, CLR), (size, size))
          # Добавляем картинку в обучающую выборку
          train_data.append(list(np.array(img)))
          # А также её индекс
          train_labels.append([pp])
    print(pp,p1)
    pp=pp+1
  
  # Формируем и выгружаем массивы
  train_labels = np.array(train_labels)
  train_data = np.array(train_data)
  return (train_data,train_labels)
  
(X_train, Y_train) = create_train_data()

0 кроссовки
1 сапоги
2 туфли


In [6]:
import keras
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

def build_model():
    # Модель последовательная...
    model = Sequential()
    
    # размером SIZE*SIZE*3. В первом блоке свёртка окном 3*3 разных окон 36 штук...
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=(size, size, 3)))
    # Активация ReLU
    model.add(Activation("relu"))
    # Подвыборка квадратом 3*3 максимума из квадрата
    model.add(MaxPooling2D(pool_size=(3, 3)))
    
    # [32, size/3, size/3]
    model.add(Conv2D(32, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # [32, size/6, size/6]
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # [64, size/12, size/12]
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # [128, size/24, size/24]
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # [64, size/48, size/48]
    model.add(Conv2D(32, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # [32, size/96, size/96]
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1024))
    model.add(Activation("relu"))
 
    # softmax classifier
    model.add(Dense(classes))
    model.add(Activation("softmax"))
 
    # return network architecture
    return model

Using TensorFlow backend.


In [0]:
from keras.utils import to_categorical

Y_train_en = to_categorical(Y_train, num_classes=classes, dtype='float32')

#from keras.utils import to_categorical
#import tensorflow as tf
#Y_train_en = tf.one_hot(Y_train, classes)

In [9]:
model = build_model()
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model.fit(X_train,Y_train_en,epochs=500, batch_size=32)

300/300 [==============================] - 16s 53ms/step - loss: 3.5700e-05 - categorical_accuracy: 1.0000
Epoch 113/500
300/300 [==============================] - 16s 53ms/step - loss: 3.5007e-05 - categorical_accuracy: 1.0000
Epoch 114/500
300/300 [==============================] - 16s 53ms/step - loss: 3.4053e-05 - categorical_accuracy: 1.0000
Epoch 115/500
128/300 [===========>..................] - ETA: 9s - loss: 2.4826e-05 - categorical_accuracy: 1.0000 

KeyboardInterrupt: ignored

In [10]:
# Скачиваем скрипт для конвертации сети
!git clone https://github.com/amir-abdi/keras_to_tensorflow.git
  
# Сохраняем сеть в формате Keras
model.save("Keras_model.h5")

# Сохраняем описание узлов в описании JSON
model_json = model.to_json()
with open("JSON_model.json", "w") as json_file:
    json_file.write(model_json)

# Сохраняем описание узлов в "замороженном" виде в формате protobuf
!python keras_to_tensorflow/keras_to_tensorflow.py --input_model="Keras_model.h5" --input_model_json="JSON_model.json" --output_model="Final_model.pb"

Cloning into 'keras_to_tensorflow'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 162 (delta 2), reused 0 (delta 0), pack-reused 156
Receiving objects: 100% (162/162), 147.06 KiB | 713.00 KiB/s, done.
Resolving deltas: 100% (79/79), done.
Using TensorFlow backend.


W1204 20:28:55.218283 140051143636864 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W1204 20:28:55.220731 140051143636864 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.


W1204 20:28:55.235162 140051143636864 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4267: The name tf.nn.max_pool is